In [1]:
from fastai.learner import *
from fastai.column_data import *
from fastai.structured import *

In [2]:
PATH=Path('../data/elo')
# C drive path
# PATH1 = Path('c:/data/')

In [3]:
# Load train customer
customer = pd.read_csv(PATH/'train.csv')

In [4]:
#get unique customer ids
customer_uniq_id = customer.groupby(['first_active_month','feature_1','feature_2','feature_3'],as_index=True).count().reset_index()
customer_uniq_id = customer_uniq_id.drop(['card_id','target'],axis=1)
customer_uniq_id['cust_index'] = customer_uniq_id.index

# customer_uniq_id.head()


In [5]:
customer_uniq_id.dtypes

first_active_month    object
feature_1              int64
feature_2              int64
feature_3              int64
cust_index             int64
dtype: object

In [9]:
len(customer_uniq_id)

495

In [10]:
customer_uniq_id['feature_1']=customer_uniq_id['feature_1'].astype(np.uint8)
customer_uniq_id['feature_3']=customer_uniq_id['feature_3'].astype(np.uint8)
customer_uniq_id['feature_2']=customer_uniq_id['feature_2'].astype(np.uint8)
customer_uniq_id['cust_index']=customer_uniq_id['cust_index'].astype(np.uint16)

In [11]:
customer_uniq_id.to_feather(f'{PATH}\customer_uniq_id')

In [14]:
# Load merchants and get unique
merchants =  pd.read_csv(PATH/'merchants.csv')
merchant_id = merchants.drop(['merchant_id','merchant_group_id','numerical_1','numerical_2','category_1','most_recent_sales_range','most_recent_purchases_range','avg_sales_lag3','avg_sales_lag6',
                              'avg_purchases_lag6','active_months_lag6','avg_sales_lag12','avg_purchases_lag12',
                              'active_months_lag12','category_4','category_2','avg_purchases_lag3','active_months_lag3'],axis=1)
merchant_id = merchant_id.groupby(['merchant_category_id','subsector_id','city_id','state_id'],as_index=True).count().reset_index()
merchant_id['merch_index']= merchant_id.index
merchant_id.head()

,merchant_category_id,subsector_id,city_id,state_id,merch_index
0,-1,-1,231,9,0
1,2,20,-1,-1,1
2,2,20,-1,1,2
3,2,20,-1,2,3
4,2,20,-1,3,4


In [15]:
print(len(merchant_id))
print(merchant_id['merchant_category_id'].max())
print(merchant_id['subsector_id'].max())
print(merchant_id['city_id'].max())
print(merchant_id['merchant_category_id'].max())

46425
891
41
347
891


In [16]:
merchant_id['merchant_category_id']=merchant_id['merchant_category_id'].astype(np.uint16)
merchant_id['subsector_id']=merchant_id['subsector_id'].astype(np.uint8)
merchant_id['city_id']=merchant_id['city_id'].astype(np.uint16)
merchant_id['state_id']=merchant_id['state_id'].astype(np.uint16)
merchant_id['merch_index']=merchant_id['merch_index'].astype(np.uint16)

In [17]:
merchant_id.to_feather(f'{PATH}\merchant_id')

In [18]:
merchant_id.head()

,merchant_category_id,subsector_id,city_id,state_id,merch_index
0,65535,255,231,9,0
1,2,20,65535,65535,1
2,2,20,65535,1,2
3,2,20,65535,2,3
4,2,20,65535,3,4


In [27]:
merchant_id[merchant_id['city_id'] == 255].count()

merchant_category_id    105571
subsector_id            105571
city_id                 105571
state_id                105571
merch_index             105571
dtype: int64

In [ ]:
#Load transactions, drop field not used
transactions= pd.read_csv(PATH/'historical_transactions.csv')
transactions1 = transactions[['card_id','merchant_category_id','subsector_id','state_id','city_id','purchase_amount','purchase_date']].copy()
del(transactions)
# transactions1.head()
# transactions1.to_feather(f'{PATH1}\transactions')

In [20]:
#merge merchants into training set
customer = pd.merge(customer,transactions1, on='card_id', how='left')
customer.to_feather(f'{PATH1}\customer')

In [21]:
#get ids for customers
customer = pd.merge(customer,customer_uniq_id,how='inner',on=['first_active_month','feature_1','feature_2','feature_3'])
customer = customer.drop(['first_active_month','feature_1','feature_2','feature_3'],axis=1)
customer.to_feather(f'{PATH1}\customer')

In [22]:
#reduce size of customers
cust_merch = customer.groupby(['cust_index','merchant_category_id','subsector_id','city_id','state_id'],as_index=False).agg({'purchase_amount' : np.sum}).reset_index().copy()
cust_merch.to_feather(f'{PATH1}\cust_merch')

In [23]:
cust_merch.head()

,index,cust_index,merchant_category_id,subsector_id,city_id,state_id,purchase_amount
0,0,0,19,36,60,16,-2.846532
1,1,0,19,36,158,15,-2.797335
2,2,0,34,38,138,15,-0.598145
3,3,0,38,7,88,16,-0.581616
4,4,0,45,18,138,15,-1.176305


In [6]:
# Load test customer
customer_test = pd.read_csv(PATH/'test.csv')

In [25]:
customer_test = pd.merge(customer_test,customer_uniq_id,how='inner',
                         on=['first_active_month','feature_1','feature_2','feature_3'])
customer_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,cust_index
0,2017-04,C_ID_0ab67a22ab,3,3,1,365
1,2017-04,C_ID_e6340286bd,3,3,1,365
2,2017-04,C_ID_7fa0d03279,3,3,1,365
3,2017-04,C_ID_2fa504fb02,3,3,1,365
4,2017-04,C_ID_967ca675f9,3,3,1,365


In [26]:
customer_test = customer_test.drop(['first_active_month','feature_1','feature_2','feature_3'],axis=1)
customer_test.to_feather(f'{PATH1}\customer_test')

In [28]:
#transfer merchant info to customer_test
len(customer_test)

123603

In [3]:
#load customer tst and merch
customer_test = pd.read_feather(f'{PATH1}\customer_test')
cust_merch = pd.read_feather(f'{PATH1}\cust_merch')

In [4]:
customer_test1 = pd.merge(customer_test,cust_merch,how='left',on=['cust_index'])
customer_test1.to_feather(f'{PATH1}\customer_test1')
customer_test1.head()

MemoryError: 

1) https://www.kaggle.com/jagangupta/memory-optimization-and-eda-on-entire-dataset 
2) https://www.kaggle.com/kunalkotian/easily-load-train-csv-w-o-crash-save-feather-file

In [25]:
mem_test=transactions1.memory_usage(index=True).sum() 

In [26]:
print("transaction dataset uses ",mem_test/ 1024**2," MB")

transaction dataset uses  666.3291397094727  MB


In [23]:
# print(transactions1.max())
# print(transactions1.min())
#check datatypes
print(transactions1.dtypes)

card_id                  object
merchant_category_id      uint8
subsector_id              uint8
state_id                  uint8
city_id                   uint8
purchase_amount         float64
Year                     uint64
Month                    uint64
Day                      uint64
dtype: object


In [10]:
transactions1['purchase_date']=pd.to_datetime(transactions1['purchase_date'],format="%Y-%m-%d")

In [14]:
transactions1['merchant_category_id'] = transactions1['merchant_category_id'].astype(np.uint8)

In [17]:
transactions1['subsector_id'] = transactions1['subsector_id'].astype(np.uint8)
transactions1['state_id'] = transactions1['state_id'].astype(np.uint8)
transactions1['city_id'] = transactions1['city_id'].astype(np.uint8)

In [20]:
transactions1['Year'] = pd.DatetimeIndex(transactions1['purchase_date']).year
transactions1['Month'] = pd.DatetimeIndex(transactions1['purchase_date']).month
transactions1['Day'] =pd.DatetimeIndex(transactions1['purchase_date']).day.astype(np.uint8)
del(transactions1['purchase_date'])

In [24]:
transactions1['Day']=transactions1['Day'].astype(np.uint8)
transactions1['Month']=transactions1['Month'].astype(np.uint8)
transactions1['Year']=transactions1['Year'].astype(np.uint16)

In [5]:
mem_cust=customer.memory_usage(index=True).sum() 
print("transaction dataset uses ",mem_cust/ 1024**2," MB")


transaction dataset uses  722.1798496246338  MB


In [39]:
print(customer.dtypes)

first_active_month       object
card_id                  object
feature_1                 uint8
feature_2                 uint8
feature_3                 uint8
target                  float64
merchant_category_id      uint8
subsector_id              uint8
state_id                  uint8
city_id                   uint8
purchase_amount         float64
Year                     uint16
Month                     uint8
Day                       uint8
dtype: object


In [30]:
customer['feature_1']=customer['feature_1'].astype(np.uint8)
customer['feature_3']=customer['feature_3'].astype(np.uint8)
customer['feature_2']=customer['feature_2'].astype(np.uint8)

In [32]:
customer = pd.merge(customer,transactions1, on='card_id', how='left')

In [35]:
customer['feature_2']=customer['feature_2'].astype(np.uint8)

In [38]:
del(customer['feature_4'])

In [15]:
customer.to_feather(f'{PATH}\customer')

In [13]:
customer = pd.read_feather(f'{PATH}\customer')

In [16]:
customer_test.to_feather(f'{PATH}\customer_test')

In [4]:
customer['Year']=customer['Year'].astype(np.uint16)

In [11]:
mem_cust_test=customer_test.memory_usage(index=True).sum() 
print("transaction dataset uses ",mem_cust_test/ 1024**2," MB")



transaction dataset uses  2.2401018142700195  MB


In [9]:
print(customer_test.dtypes)

first_active_month    object
card_id               object
feature_1              int64
feature_2              int64
feature_3              int64
dtype: object


In [10]:
customer_test['feature_1']=customer_test['feature_1'].astype(np.uint8)
customer_test['feature_3']=customer_test['feature_3'].astype(np.uint8)
customer_test['feature_2']=customer_test['feature_2'].astype(np.uint8)

In [20]:
customer.head()

,card_id,target,merchant_category_id,subsector_id,state_id,city_id,purchase_amount,Year,Month,Day,cust_index
0,C_ID_92a2005557,-0.820283,195,34,9,69,-0.506033,225,11,25,142
1,C_ID_92a2005557,-0.820283,22,37,9,69,-0.722114,225,7,16,142
2,C_ID_92a2005557,-0.820283,171,34,9,69,-0.726171,225,8,10,142
3,C_ID_92a2005557,-0.820283,53,21,9,69,-0.610167,225,12,15,142
4,C_ID_92a2005557,-0.820283,22,37,9,69,-0.307232,225,7,8,142


In [15]:
customer = pd.merge(customer,customer_uniq_id,how='inner',on=['first_active_month','feature_1','feature_2','feature_3'])

In [19]:
customer.drop(['first_active_month','feature_1','feature_2','feature_3'],axis=1,inplace=True)

In [23]:
customer.to_feather(f'{PATH}\customer_id')

In [21]:
customer = pd.merge(customer,merchant_id,how='inner',on=['merchant_category_id','subsector_id','state_id','city_id'])

MemoryError: 

In [12]:
customer_test = pd.merge(customer_test,customer, how='inner',on=['feature_1','feature_2','feature_3'])

MemoryError: 